<a href="https://colab.research.google.com/github/AliEdalat/ChatBot_for_persian_LLaMA_fine_tune/blob/main/ChatBot_for_persian_LLaMA_fine_tune.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# # Install FastChat
# !pip3 install fschat

# # Install the latest main branch of huggingface/transformers
# !pip3 install git+https://github.com/huggingface/transformers

In [2]:
!git clone https://github.com/lm-sys/FastChat.git
%cd FastChat

fatal: destination path 'FastChat' already exists and is not an empty directory.
/content/FastChat


In [3]:
# !pip install transformers

In [4]:
!pip3 install --upgrade pip  # enable PEP 660 support
!pip3 install -e .

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Obtaining file:///content/FastChat
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Installing backend dependencies ... done
  Preparing editable metadata (pyproject.toml) ... done
  Cloning https://github.com/huggingface/transformers.git to /tmp/pip-install-8pwwot5x/transformers_5bbe196f8c6243318ec3b3f2aa6cef88
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers.git /tmp/pip-install-8pwwot5x/transformers_5bbe196f8c6243318ec3b3f2aa6cef88
  Resolved https://github.com/huggingface/transformers.git to commit 656e869a4523f6a0ce90b3aacbb05cc8fb5794bb
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparin

In [5]:
!pip install einops
!pip install peft
# !pip install flash-attn

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [6]:
# bf16 and tf32 for new gpu, per_node = num of gpu

In [17]:
!torchrun --nnodes=1 --nproc_per_node=1 --master_port=4545 \
    fastchat/train/train_mem.py \
    --model_name_or_path 'decapoda-research/llama-7b-hf' \
    --data_path './playground/data/alpaca-data-conversation.json' \
    --bf16 False \
    --output_dir ./checkpoints \
    --num_train_epochs 3 \
    --per_device_train_batch_size 1 \
    --per_device_eval_batch_size 4 \
    --gradient_accumulation_steps 1 \
    --evaluation_strategy "no" \
    --save_strategy "steps" \
    --save_steps 1200 \
    --save_total_limit 100 \
    --learning_rate 2e-5 \
    --weight_decay 0. \
    --warmup_ratio 0.03 \
    --lr_scheduler_type "cosine" \
    --logging_steps 1 \
    --fsdp "full_shard auto_wrap" \
    --fsdp_transformer_layer_cls_to_wrap 'LlamaDecoderLayer' \
    --tf32 False \
    --model_max_length 2048 \
    --gradient_checkpointing True \
    --lazy_preprocess True

2023-04-09 19:40:52.334185: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
╭───────────────────── Traceback (most recent call last) ──────────────────────╮
│ /content/FastChat/fastchat/train/train_mem.py:5 in <module>                  │
│                                                                              │
│    2 # Make it more memory efficient by monkey patching the LLaMA model with │
│    3                                                                         │
│    4 # Need to call this before importing transformers.                      │
│ ❱  5 from fastchat.train.llama_flash_attn_monkey_patch import replace_llama_ │
│    6                                                                         │
│    7 replace_llama_attn_with_flash_attn()                                    │
│    8                                                                         │
│                                                                 

In [18]:
# !cat /usr/local/lib/python3.9/dist-packages/transformers/training_args.py

In [7]:
from transformers import MT5ForConditionalGeneration, MT5Tokenizer

model_size = "large"
model_name = f"persiannlp/mt5-{model_size}-parsinlu-translation_en_fa"
tokenizer = MT5Tokenizer.from_pretrained(model_name)
model = MT5ForConditionalGeneration.from_pretrained(model_name)



def run_MT_model(input_string, **generator_args):
    generator_args = {
    "max_length": 256,
    "num_beams": 4,
    "length_penalty": 1.5,
    "no_repeat_ngram_size": 3,
    "early_stopping": True,
    }
    input_ids = tokenizer.encode(input_string, return_tensors="pt")
    res = model.generate(input_ids, **generator_args)
    output = tokenizer.batch_decode(res, skip_special_tokens=True)
    print(output)
    return output

In [10]:
# _ = run_MT_model("i am disabled pregnant in iran. i want to follow iran instructions. it is very nice but it is lazy")
# run_MT_model("shrouds herself in white and walks penitentially disguised as brotherly love through factories and parliaments; offers help, but desires power;")
# run_MT_model("He thanked all fellow bloggers and organizations that showed support.")
# run_MT_model("Races are held between April and December at the Veliefendi Hippodrome near Bakerky, 15 km (9 miles) west of Istanbul.")
run_MT_model("I want to pursue PhD in Computer Science about social network,what is the open problem in social networks?")

['من می خواهم پایان نامه دکتری را در رشته علوم کامپیوتر در مورد شبکه اجتماعی دنبال کنم، مشکل باز در شبکه های اجتماعی چیست؟']


['من می خواهم پایان نامه دکتری را در رشته علوم کامپیوتر در مورد شبکه اجتماعی دنبال کنم، مشکل باز در شبکه های اجتماعی چیست؟']